In [1]:
%cd /Users/natalipeeva/Documents/GitHub/Automatic-Answering-of-City-Council-Questions/

/Users/natalipeeva/Documents/GitHub/Automatic-Answering-of-City-Council-Questions


In [2]:
import sys
import os
src_dir = os.path.join(os.getcwd(), 'src')
sys.path.append(src_dir)

from read_data.read_data import read_urls_questions, get_questions, get_url_content_tuples, get_relevant_docs
from elasticsearch import Elasticsearch
from retrieval.sparse_retrieval.bm25 import set_index, get_result_tuples

In [3]:
es_client = Elasticsearch("http://localhost:9200")

In [4]:
es_client.info().body

{'name': 'c913d73a452f',
 'cluster_name': 'docker-cluster',
 'cluster_uuid': '2OYk7KTMQx6zkjPCEjNf2Q',
 'version': {'number': '8.7.0',
  'build_flavor': 'default',
  'build_type': 'docker',
  'build_hash': '09520b59b6bc1057340b55750186466ea715e30e',
  'build_date': '2023-03-27T16:31:09.816451435Z',
  'build_snapshot': False,
  'lucene_version': '9.5.0',
  'minimum_wire_compatibility_version': '7.17.0',
  'minimum_index_compatibility_version': '7.0.0'},
 'tagline': 'You Know, for Search'}

In [5]:
collected, questions =  read_urls_questions('data/all_references/reference_urls_collected_clean_pdf.csv',
                                            'data/all_references/questions_updated_urls.csv', clean_url_nan=True) # read collected urls and questions + remove unsuccessful collection

In [31]:
print(len(collected))
print(len(questions))

538
451


In [6]:
question_list = get_questions(questions)
document_list = get_url_content_tuples(collected)
mappings = {
        "properties": {
            "url": {"type": "text"},
            "text": {"type": "text", "analyzer": "standard", "similarity": "BM25"}
    }
}

In [7]:
es_client = set_index(es_client=es_client, collected=collected, mappings=mappings) # 7secs

In [21]:
results = get_result_tuples(es_client=es_client, questions=question_list, n=5) # 1:45 min

In [22]:
from irmetrics.topk import recall
predictions = []
for question in results.keys():
    urls = []
    for result in results[question][1]:
        urls.append(result['url'])
    predictions.append(list(set(urls)))


relevant_docs = get_relevant_docs(questions)

true = []
for question in relevant_docs.keys():
    true.append(list(set(relevant_docs[question])))

In [41]:
import numpy as np
from irmetrics.topk import rr

# Calculate the Mean Reciprocal Rank for each question
mrr_values = []
for i in range(len(predictions)):
    true_values = true[i]
    mrr = rr(true_values, predictions[i], k =5)
    mrr_values.append(mrr)

# Calculate the average Mean Reciprocal Rank
average_mrr = np.mean(mrr_values)

print("Average Mean Reciprocal Rank:", average_mrr)


Average Mean Reciprocal Rank: 0.13560775540641312


In [36]:
from irmetrics.topk import recall

# Calculate the Mean Reciprocal Rank for each question
mrr_values = []
for i in range(len(predictions)):
    true_values = true[i]
    mrr = recall(true_values, predictions[i], k=5)
    mrr_values.append(mrr)

# Calculate the average Mean Reciprocal Rank
average_recall = np.mean(mrr_values)

print("Average Mean Reciprocal Rank:", average_recall)

Average Mean Reciprocal Rank: 0.22408650260999252


In [42]:
import pickle
with open ('data/results/bm25_results_all_ref.pickle', 'wb') as f:
    pickle.dump(results, f)

In [43]:
questions.to_csv('data/question_answer/all_ref_questions.csv')